Bad model. Unfair. Optimized in a ML sense


In [16]:
import os, json, random, shutil
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, average_precision_score

rstate = 1
target = "checked"
GENDER_COL = "persoon_geslacht_vrouw" # nt used?

data_path = "data/investigation_train_large_checked.csv"

df = pd.read_csv(data_path)
print("Loaded:", data_path, "shape:", df.shape)

y = df[target].astype(int).values

drop_cols = [target, "Ja", "Nee"]

X = df.drop(columns=drop_cols)

print("Features:", X.shape[1], "Positive rate:", y.mean().round(4))

Loaded: data/investigation_train_large_checked.csv shape: (130000, 318)
Features: 315 Positive rate: 0.15


In [17]:
# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=rstate, stratify=y
)
X_train.shape, X_test.shape

((104000, 315), (26000, 315))

## Model choice (Bad model)

We use **GradientBoostingClassifier** because it:
- captures **non-linear interactions** and can exploit proxy/leakage patterns,
- performs well on mixed binary/dummy features without extra preprocessing,
- converts cleanly to ONNX with `skl2onnx`.


In [18]:
#HYper param tuning

gb = GradientBoostingClassifier(random_state=rstate)

param_dist = {
    "n_estimators": [100, 150, 250],
    "learning_rate": [0.25, 0.5, 0.75, 1],
    "max_depth": [1, 3],
    "subsample": [1.0],
    "min_samples_leaf": [3],
}

search = RandomizedSearchCV(
    gb,
    param_distributions=param_dist,
    n_iter=6,          
    scoring="roc_auc",
    cv=2,           
    random_state=rstate,
    n_jobs=-1,
    verbose=1
)
search.fit(X_train, y_train)

bad_model = search.best_estimator_
search.best_params_, search.best_score_

C:\Users\alexa\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_search.py:317: UserWarning: The total space of parameters 5 is smaller than n_iter=6. Running 5 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fitting 2 folds for each of 5 candidates, totalling 10 fits


({'subsample': 1.0,
  'n_estimators': 100,
  'min_samples_leaf': 3,
  'max_depth': 3,
  'learning_rate': 0.5},
 np.float64(0.9858320394131859))

In [19]:
# Evaluate (classical metrics)
p = bad_model.predict_proba(X_test)[:, 1]
yhat = (p >= 0.5).astype(int)

acc = accuracy_score(y_test, yhat)
roc = roc_auc_score(y_test, p)
prauc = average_precision_score(y_test, p)

print(f"BAD model — Accuracy: {acc:.4f}")
print(f"BAD model — ROC-AUC:  {roc:.4f}")
print(f"BAD model — PR-AUC:   {prauc:.4f}")
# BAD model — Accuracy: 0.9530
# BAD model — ROC-AUC:  0.9863
# BAD model — PR-AUC:   0.9343


BAD model — Accuracy: 0.9530
BAD model — ROC-AUC:  0.9863
BAD model — PR-AUC:   0.9343


## Why these metrics?
- **Accuracy**: intuitive baseline; can look good even when the model is unfair.
- **ROC-AUC**: ranking quality across thresholds; common for fraud-like tasks.
- **PR-AUC**: focuses on the positive class; more informative when positives are rarer.

These are “classical ML” metrics that may not reveal discrimination by themselves.


In [20]:
# Export to ONNX
onnx_tmp = "model/bad_model_tmp.onnx"
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType

initial_type = [("float_input", FloatTensorType([None, X_train.shape[1]]))]
onnx_model = convert_sklearn(bad_model, initial_types=initial_type)

with open(onnx_tmp, "wb") as f:
    f.write(onnx_model.SerializeToString())

print("Saved temporary ONNX:", onnx_tmp)

Saved temporary ONNX: model/bad_model_tmp.onnx
